In [1]:
!wget https://raw.githubusercontent.com/sjut/HSE-Compling/master/seminars/data/reviews_tok.txt
!wget https://raw.githubusercontent.com/sjut/HSE-Compling/master/seminars/data/reviews_scores.txt

--2019-12-23 21:54:20--  https://raw.githubusercontent.com/sjut/HSE-Compling/master/seminars/data/reviews_tok.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8489426 (8.1M) [text/plain]
Saving to: ‘reviews_tok.txt’

reviews_tok.txt     100%[===================>]   8.10M  42.7MB/s    in 0.2s    

2019-12-23 21:54:22 (42.7 MB/s) - ‘reviews_tok.txt’ saved [8489426/8489426]

--2019-12-23 21:54:24--  https://raw.githubusercontent.com/sjut/HSE-Compling/master/seminars/data/reviews_scores.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 373

In [89]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
import collections
import math
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
texts = []
with open("reviews_tok.txt") as f:
    for line in f:
        texts.append(line.rstrip('\r\n'))
scores = []
with open("reviews_scores.txt") as f:
    scores = list(map(lambda x: float(x.rstrip("\r\n")), f.readlines()))

In [0]:
pos_texts = []
neg_texts = []
pos_array = []
neg_array = []

In [0]:
def preproc_data(text, score):
  text_score = {}
  for i in range(len(text)):
    text_score[text[i]] = score[i]

  for value in text_score:
    if text_score[value] >= 8:
      pos_texts.append(value)
    elif text_score[value] <= 2:
      neg_texts.append(value)

  for i in pos_texts:
    i = i.split()
    for w in i:
      if w[-4:] == 'VERB' or w[-4:] == 'ADV' or w[-3:] == 'ADJ':
        w = w.split('_')
        if w[0] not in russian_stopwords and w[0] != 'мочь':
          pos_array.append(w[0])

  for e in neg_texts:
    e = e.split()
    for r in e:
      if r[-4:] == 'VERB' or r[-4:] == 'ADV' or r[-3:] == 'ADJ':
        r = r.split('_')
        if r[0] not in russian_stopwords and w[0] != 'мочь':
          neg_array.append(r[0])

In [0]:
preproc_data(texts, scores)

In [0]:
pos_vocab = collections.Counter()
for word in pos_array:
  pos_vocab[word] += 1

In [0]:
neg_vocab = collections.Counter()
for word in neg_array:
  neg_vocab[word] += 1

In [0]:
difference = {}

In [74]:
difference

{2.32008654970587e-07: ('выйти',
  'НЕГ',
  0.0008565310492505353,
  'ПОЗ',
  0.0008562990405955647),
 3.962857509901127e-06: ('получиться',
  'НЕГ',
  0.0014989293361884369,
  'ПОЗ',
  0.001502892193698338),
 4.426874819842463e-06: ('посыпать',
  'НЕГ',
  0.00021413276231263382,
  'ПОЗ',
  0.00020970588749279136),
 8.853749639684927e-06: ('обнаружить',
  'НЕГ',
  0.00042826552462526765,
  'ПОЗ',
  0.0004194117749855827),
 9.549775604596796e-06: ('прийтись',
  'НЕГ',
  0.0029978586723768737,
  'ПОЗ',
  0.002988308896772277),
 1.2816607149586162e-05: ('менять',
  'НЕГ',
  0.0010706638115631692,
  'ПОЗ',
  0.0010834804187127554),
 1.304861580455683e-05: ('дальний',
  'НЕГ',
  0.00021413276231263382,
  'ПОЗ',
  0.00022718137811719065),
 1.351263311449814e-05: ('обычный',
  'НЕГ',
  0.0014989293361884369,
  'ПОЗ',
  0.0014854167030739387),
 1.747549062439928e-05: ('неискушённый',
  'НЕГ',
  0.0,
  'ПОЗ',
  1.747549062439928e-05),
 2.190236544424173e-05: ('рисовать',
  'НЕГ',
  0.0002141327

In [70]:
neg_vocab.most_common(10)

[('сказать', 67),
 ('вкусный', 62),
 ('мочь', 56),
 ('понравиться', 53),
 ('приятный', 47),
 ('принести', 43),
 ('хороший', 39),
 ('большой', 37),
 ('хотеться', 32),
 ('заказать', 31)]

In [0]:
for i in pos_vocab:
  difference[math.fabs((pos_vocab[i]/len(pos_array))-(neg_vocab[i]/len(neg_array)))] = (i, 'НЕГ',neg_vocab[i]/len(neg_array), 'ПОЗ',pos_vocab[i]/len(pos_array))

In [0]:
def sentim_lex(vocab):
  sentim_vocab = {}
  sort = sorted(vocab.items(), reverse = True)
  for elem in sort[:300]:
    if elem[1][2] > elem[1][4]:
      sentim_vocab[elem[1][0]] = 0
    else:
      sentim_vocab[elem[1][0]] = 1
  return sentim_vocab

In [0]:
final_vocab = sentim_lex(difference)

In [77]:
print(final_vocab)

{'сказать': 0, 'хороший': 1, 'решить': 0, 'стоить': 0, 'отмечать': 0, 'итальянский': 0, 'предложить': 0, 'видеть': 0, 'принести': 0, 'первый': 1, 'милый': 0, 'хотеться': 0, 'звать': 0, 'домашний': 0, 'радовать': 0, 'думать': 0, 'забыть': 0, 'интересный': 1, 'данный': 0, 'нравиться': 1, 'жить': 0, 'слышать': 0, 'самый': 1, 'говорить': 1, 'подавать': 1, 'алкогольный': 0, 'пить': 1, 'давать': 0, 'идти': 0, 'оказаться': 1, 'молодой': 1, 'подобный': 0, 'поесть': 0, 'второй': 1, 'понять': 1, 'просить': 0, 'должен': 1, 'смотреть': 1, 'открыть': 0, 'ответить': 0, 'ненавязчивый': 0, 'отдельный': 0, 'рабочий': 0, 'оценить': 0, 'светлый': 0, 'постоянный': 0, 'посетить': 1, 'свежий': 1, 'писать': 0, 'уютный': 0, 'понимать': 1, 'русский': 1, 'сесть': 1, 'средний': 1, 'больший': 1, 'дожидаться': 0, 'обслуживать': 0, 'бывать': 0, 'напомнить': 0, 'Казанский': 0, 'приличный': 1, 'платить': 0, 'австрийский': 0, 'правильный': 1, 'напоминать': 0, 'сложный': 0, 'неплохой': 0, 'мокрый': 0, 'острый': 1, 'дов

In [0]:
food = open('Food_words.txt', 'r')
food = food.readlines()

In [0]:
service = open('Service_words.txt', 'r')
service = service.readlines()

In [0]:
def add(file):
  for wo in file:
    wo = wo.split('\t')
    #print(wo[1], wo[2][:-1], type(wo[2]))
    if wo[1] in final_vocab and wo[2][:-1]==str(final_vocab[wo[1]]):
      print(wo[1], final_vocab[wo[1]])
    elif wo[1] in final_vocab and wo[2][:-1] != str(final_vocab[wo[1]]):
      final_vocab[wo[1]] = wo[2][:-1]
    elif wo[1] not in final_vocab:
      final_vocab[wo[1]] = wo[2][:-1]

In [82]:
add(service)

вежливый 1
хороший 1
отличный 1


In [84]:
add(food)

достойный 1
отличный 1
сытный 1
свежий 1
интересный 1
необычный 1
приятный 1


In [0]:
with open('vocab1.pickle', 'wb') as f:
        pickle.dump(final_vocab, f)

In [0]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(pos_texts)

In [0]:
results_df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())

In [91]:
dic = {}

for c in vectorizer.get_feature_names():
    dic[sum(results_df[c])] = c
result = sorted(dic.items(), reverse = True)

print(result[:200])

[(812.7932097224274, '_punct'), (102.19224010902649, 'и_cconj'), (93.41550278524655, 'в_adp'), (86.9631465561488, 'не_part'), (62.24488957966751, 'на_adp'), (60.51752668350102, 'с_adp'), (51.49329395515203, 'быть_aux'), (51.1411133080893, 'очень_adv'), (50.905489281521966, 'я_pron'), (49.005486653478776, 'мы_pron'), (41.09073004616673, 'но_cconj'), (39.45258964987203, 'что_sconj'), (35.709615616268074, 'ресторан_noun'), (33.8557527201622, 'все_pron'), (32.46607097439565, 'а_cconj'), (30.253986228498988, 'это_pron'), (29.71390704188227, 'и_part'), (28.102666483813255, 'за_adp'), (27.63740969137582, 'из_adp'), (27.587254983678246, 'по_adp'), (26.99969912400313, 'как_sconj'), (26.81973676388736, 'быть_verb'), (26.76646962619762, 'этот_det'), (26.437392808187997, 'место_noun'), (26.42460304639622, 'вкусный_adj'), (26.20673719834817, 'так_adv'), (25.691752454210345, 'для_adp'), (25.489595148847766, 'кухня_noun'), (25.06920121082889, 'хороший_adj'), (25.00890669898562, 'что_pron'), (23.72639

In [0]:
vectorizer = TfidfVectorizer()
Y = vectorizer.fit_transform(neg_texts)
results_df_n = pd.DataFrame(Y.toarray(), columns = vectorizer.get_feature_names())

In [93]:
dic_neg = {}

for v in vectorizer.get_feature_names():
    dic_neg[sum(results_df_n[v])] = v
result_n = sorted(dic_neg.items(), reverse = True)
print(result_n[:200])

[(76.45992667418662, '_punct'), (10.196492902073704, 'и_cconj'), (8.77972716226851, 'в_adp'), (8.444015350042635, 'не_part'), (5.737054203899289, 'на_adp'), (5.49475943946468, 'с_adp'), (5.406263705996067, 'быть_aux'), (5.214853268697763, 'я_pron'), (5.099991157244742, 'мы_pron'), (5.066565098341079, 'очень_adv'), (4.0799389460175846, 'но_cconj'), (3.9131823808322928, 'все_pron'), (3.7207830161984075, 'ресторан_noun'), (3.585034118798328, 'что_sconj'), (3.231749996414143, 'это_pron'), (3.1427964841802254, 'за_adp'), (2.8291009531630973, 'а_cconj'), (2.733030260849939, 'что_pron'), (2.722920018098609, 'из_adp'), (2.6755594358478048, 'быть_verb'), (2.5786404794631737, 'место_noun'), (2.5739961028790965, 'по_adp'), (2.5706141675167156, 'вкусный_adj'), (2.5194073346316768, 'этот_det'), (2.4169261761155747, 'и_part'), (2.407300241470524, 'раз_noun'), (2.383049213219033, 'официант_noun'), (2.3592165615191742, 'так_adv'), (2.2415886748290856, 'обслуживание_noun'), (2.179024738637183, 'весь_de

In [0]:
res_n = []
for r in result_n:
  res_n.append(r[1])
res_p = []
for e in result:
  res_p.append(e[1])

In [0]:
neg_tokens = []
for n in res_n:
  if n not in res_p:
    neg_tokens.append(n)

In [96]:
neg_tokens

['суперрррр_noun',
 'себа_noun',
 'маргаритт_propn',
 'ха',
 'корчить_verb',
 'проникнуться_verb',
 'заказ_adv',
 'мамалыг_propn',
 'саквояж_propn',
 'снэч_noun',
 'аида_propn',
 'соление_noun',
 'корпаротив_noun',
 'стёкл_noun',
 'улучшать_verb',
 'стружка_noun',
 'фыркий_adj',
 'парить_verb',
 'метакс_noun',
 'фонд_noun',
 'х_noun',
 'продолжительный_adj',
 'суперый_noun',
 'достойный_adv',
 'фомильярность_noun',
 'арам_propn',
 'эскалибур_propn',
 'мама_propn',
 'шопинг_noun',
 'ремонтировать_verb',
 '26',
 'солнце_propn',
 'эрмитажа_propn',
 'чудесно_adj',
 'гуляшный_adj',
 'разжевывать_verb',
 'универ_propn',
 'шотер_noun',
 'операция_noun',
 'издевательство_noun',
 'энгелься_verb',
 'развертываться_verb',
 'сингапур_propn',
 'устранить_verb',
 'некорректный_adj',
 'элегантность_noun',
 'штирлиц_noun',
 'рома_propn',
 'уменьшиться_verb',
 'мой_num',
 'трк_propn',
 'янтарь_noun',
 'чизкейкий_adj',
 'тамаринд_noun',
 'экспонат_noun',
 'хм',
 'энергетик_noun',
 'карпачо_noun',
 'хоро

In [0]:
pos_tokens = []

In [0]:
for i in result:
  if i[1] not in res_n:
    pos_tokens.append(i[1])

In [99]:
pos_tokens

['_x',
 'иногда_adv',
 'выходной_noun',
 'веселый_adj',
 'постоянно_adv',
 'русский_adj',
 'программа_noun',
 'рекомендой_adj',
 'чудесный_adj',
 'масло_noun',
 'клуб_noun',
 'китайский_adj',
 'высота_noun',
 'начало_noun',
 'вовремя_adv',
 'утро_noun',
 'во',
 'претензия_noun',
 'жизнь_noun',
 'бутылка_noun',
 'месяц_noun',
 'доброжелательный_adj',
 'администрация_noun',
 'жена_noun',
 'стейка_noun',
 'больший_adj',
 'кушать_verb',
 'вежливо_adv',
 'баклажан_noun',
 'оформить_verb',
 'водка_noun',
 'располагать_verb',
 'барный_adj',
 'закрыть_verb',
 'пригласить_verb',
 'демократичный_adj',
 'почему',
 'целый_adj',
 'процветание_noun',
 'восточный_adj',
 'звонить_verb',
 'все',
 'недостаток_noun',
 'винный_adj',
 'нога_noun',
 'футбол_noun',
 'меняться_verb',
 'имя_noun',
 'всего_part',
 'приготовление_noun',
 'симпатичный_adj',
 'мясной_adj',
 'супчик_noun',
 'везде_adv',
 'острый_adj',
 'вывод_noun',
 'чисто_adv',
 'лососить_verb',
 'приемлемый_adj',
 'рыбный_adj',
 'поразить_verb',